<a href="https://colab.research.google.com/github/iamdsc/text_summarization/blob/master/text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Extractive Text Summarization 

In [1]:
# importing required libraries
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
import re
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# getting data from google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd ..
%cp content/drive/My\ Drive/dataset.zip content/

In [16]:
%cd content

/content


In [0]:
!unzip dataset.zip

In [23]:
# Reading in the training data
import os

X_text = []
counter = 0
file_list = os.listdir('dataset/stories_text_summarization_dataset_train/')
for filename in file_list:
  counter+=1
  with open(os.path.join('dataset/stories_text_summarization_dataset_train/',filename), 'r', encoding='utf-8') as f:
    content=f.read()
    X_text.append(content)
    f.close()
  if counter % 10000 == 0:
    print(counter)

10000
20000
30000
40000
50000
60000
70000
80000
90000


### Remove punctuation,numbers and special characters and make lowercase

In [0]:
def remove_punc(sent):
  """ To remove punctuations and special characters """
  cleaned=re.sub(r'[?|!|\'|"|#]', r'', sent)
  cleaned=re.sub(r'[.|,|)|(|\|/]', r' ', sent)
  cleaned=cleaned.strip()
  cleaned=cleaned.replace('\n',' ')
  return cleaned
 
def keep_alpha(sent):
  """ To remove non-alphabetic characters """
  alpha_sent=''
  for word in sent.split():
    alpha_word=re.sub('[^a-z A-Z]+', ' ', word)
    alpha_sent+=alpha_word
    alpha_sent+=' '
  alpha_sent=alpha_sent.strip()
  return alpha_sent

X_text_clean = [remove_punc(sent) for sent in X_text]
X_text_clean = [keep_alpha(sent) for sent in X_text_clean]
X_text_clean = [sent.lower() for sent in X_text_clean]

### Removing stop words

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [0]:
# function to remove stopwords
def remove_stopwords(sent):
  sent_new = ' '.join([i for i in sent if i not in stop_words])
  return sent_new

In [0]:
X_text_clean = [remove_stopwords(r.split()) for r in X_text_clean]

In [34]:
print('No. of sentences',len(X_text_clean))
print('First 5 sentences',X_text_clean[:5])

No. of sentences 92569
First 5 sentences ['cnn mac user years since made switch never thought day would come say excited next version windows fall microsoft launch windows radical redesign windows grown accustomed since windows made debut years ago windows reimagined ground built run traditional desktop touchscreen tablets first glance notice classic start menu gone replaced gorgeous tile layout full favorite apps files microsoft calls metro interface imagine blown version windows phone interface used several smartphones little tiles provide live updates news sports weather twitter mentions facebook updates without even open app one quick glance get information need old windows makes excited try final version windows fall un microsoft move company created radical redesign desktop well decade incredibly functional beta version windows available free consumer preview using last weeks right handful prototype tablets run windows using laptop instead however since metro interface designed t

In [36]:
## Splitting sentences into words for building word embeddings
X_sent_words = []
for sent in X_text_clean:
  sent_word_list = [word for word in sent.split()]
  X_sent_words.append(sent_word_list)
print(X_sent_words[:5])

[['cnn', 'mac', 'user', 'years', 'since', 'made', 'switch', 'never', 'thought', 'day', 'would', 'come', 'say', 'excited', 'next', 'version', 'windows', 'fall', 'microsoft', 'launch', 'windows', 'radical', 'redesign', 'windows', 'grown', 'accustomed', 'since', 'windows', 'made', 'debut', 'years', 'ago', 'windows', 'reimagined', 'ground', 'built', 'run', 'traditional', 'desktop', 'touchscreen', 'tablets', 'first', 'glance', 'notice', 'classic', 'start', 'menu', 'gone', 'replaced', 'gorgeous', 'tile', 'layout', 'full', 'favorite', 'apps', 'files', 'microsoft', 'calls', 'metro', 'interface', 'imagine', 'blown', 'version', 'windows', 'phone', 'interface', 'used', 'several', 'smartphones', 'little', 'tiles', 'provide', 'live', 'updates', 'news', 'sports', 'weather', 'twitter', 'mentions', 'facebook', 'updates', 'without', 'even', 'open', 'app', 'one', 'quick', 'glance', 'get', 'information', 'need', 'old', 'windows', 'makes', 'excited', 'try', 'final', 'version', 'windows', 'fall', 'un', 'mi

In [0]:
## training word2vec model to create word embeddings
model = Word2Vec(X_sent_words, min_count=1)

In [0]:
## saving the model
model.wv.save_word2vec_format('word_embeddings.bin')

In [39]:
# Generating Summary for a test files
X_test = []
counter = 0
file_list = os.listdir('dataset/stories_text_summarization_dataset_test/')
for filename in file_list:
  counter+=1
  with open(os.path.join('dataset/stories_text_summarization_dataset_test/',filename), 'r', encoding='utf-8') as f:
    content=f.read()
    X_test.append(content)
    f.close()
  print(counter)

1
2
3
4
5
6
7
8
9
10


In [0]:
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

In [0]:
def summarize(content, num_sentences):
  # splitting into sentences
  sentences = sent_tokenize(content)
  
  # cleaning sentences as above
  sentences_clean = [remove_punc(sent) for sent in sentences]
  sentences_clean = [keep_alpha(sent) for sent in sentences_clean]
  sentences_clean = [sent.lower() for sent in sentences_clean]
  
  # removing stopwords
  sentences_clean = [remove_stopwords(r.split()) for r in sentences_clean]
    
  # making sentence vectors
  sentence_vectors = []
  for i in sentences_clean:
    if len(i)!=0:
      v = sum([model[w] for w in i.split()])/(len(i.split())+0.001)
    else:
      v = np.zeros((100,))
    sentence_vectors.append(v)
  
  # similarity matrix
  sim_mat = np.zeros([len(sentences), len(sentences)])
  
  for i in range(len(sentences)):
    for j in range(len(sentences)):
      if i!=j:
        sim_mat[i][j]=cosine_similarity(sentence_vectors[i].reshape(1,100),
                                       sentence_vectors[j].reshape(1,100))[0,0]
  
  # building graph for ranking
  nx_graph = nx.from_numpy_array(sim_mat)
  scores = nx.pagerank(nx_graph)
  
  ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
  
  # Generate summary
  summary=[]
  for i in range(num_sentences):
    summary.append(ranked_sentences[i][1])
  
  return summary

In [50]:
print(X_test[0])
print('')
print('Summary')

summary = summarize(X_test[0],5)
for sent in summary:
  print(sent)

The man suspected of trying to attack the Dutch royal family by crashing his car near their bus has died, Dutch police said Friday.

A car is pictured after crashing into the crowd waiting for the visit of the royal family in Apeldoorn.

The man, whose name was not released, had been seriously injured in the crash Thursday in the town of Apeldoorn, about 45 miles east of Amsterdam, police said. He died early Friday, police said.

He had been charged with trying to attack the royal family, authorities said. Were you there? Send us your video, images

Police searched the man's house but found no weapons, explosives, "or any other clues that could lead to the involvement of other people," police said.

Five people, as well as the driver, were killed and 12 were wounded in the incident, which happened during the country's annual Queen's Day celebration, police said.

Crowds had lined the streets to see Queen Beatrix and her family ride by in an open-top bus during the Netherlands' annual h

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
